In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import warnings
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 100)
warnings.filterwarnings('ignore')

In [2]:
train_label = pd.read_hdf('C:/Users/f3107/Desktop/hy_data/train_204.h5')
test_label = pd.read_hdf('C:/Users/f3107/Desktop/hy_data/test_204.h5')

In [3]:
train_label['type'].value_counts(1)

拖网    0.623000
围网    0.231571
刺网    0.145429
Name: type, dtype: float64

In [4]:
#LabelEncoder
type_map = dict(zip(train_label['type'].unique(), np.arange(3)))
type_map_rev = {v:k for k,v in type_map.items()}
train_label['type'] = train_label['type'].map(type_map)

In [5]:
features = [x for x in train_label.columns if x not in ['ship','x','y','v','d','datetime','type','t','d_d','d_t',
                                                        'd_x','v_x','d_y','v_y','hour','date','diff_time']]
target = 'type'

In [6]:
print(len(features), ','.join(features))

52 x_max,x_min,x_mean,x_std,x_skew,x_sum,x_median,y_max,y_min,y_mean,y_std,y_skew,y_sum,y_median,v_max,v_min,v_mean,v_std,v_skew,v_sum,v_median,x_max_x_min,y_max_y_min,y_max_x_min,x_max_y_min,slope_1,slope_2,area,mode_hour,v_cut_0.5_count_0.5,v_cut_0.5_count_1.0,v_cut_0.5_count_1.5,v_cut_0.5_count_2.0,v_cut_0.5_count_2.5,v_cut_0.5_count_3.0,v_cut_0.5_count_3.5,v_cut_0.5_count_4.0,v_cut_0.5_count_4.5,v_cut_0.5_count_5.0,v_cut_0.5_count_5.5,v_cut_0.5_count_6.0,v_cut_0.5_count_6.5,v_cut_0.5_count_7.0,v_cut_0.5_count_7.5,v_cut_0.5_count_8.0,v_cut_0.5_count_8.5,v_cut_0.5_count_9.0,v_cut_0.5_count_9.5,v_cut_0.5_count_10.0,v_cut_0.5_count_20.0,v_cut_tuo_count_6,v_cut_tuo_count_20


In [7]:
params = {
    'n_estimators': 5000,
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 3,
    'early_stopping_rounds': 100,
}

In [8]:
fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

X = train_label[features].copy()
y = train_label[target]
models = []
pred = np.zeros((len(test_label),3))
oof = np.zeros((len(X), 3))
for index, (train_idx, val_idx) in enumerate(fold.split(X, y)):

    train_set = lgb.Dataset(X.iloc[train_idx], y.iloc[train_idx])
    val_set = lgb.Dataset(X.iloc[val_idx], y.iloc[val_idx])

    model = lgb.train(params, train_set, valid_sets=[train_set, val_set], verbose_eval=100)
    models.append(model)
    val_pred = model.predict(X.iloc[val_idx])
    oof[val_idx] = val_pred
    val_y = y.iloc[val_idx]
    val_pred = np.argmax(val_pred, axis=1)
    print(index, 'val f1', metrics.f1_score(val_y, val_pred, average='macro'))
    # 0.8695539641133697
    # 0.8866211724839532

    test_pred = model.predict(test_label[features])
    pred += test_pred/5

Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.0550494	valid_1's multi_logloss: 0.220085
[200]	training's multi_logloss: 0.0110202	valid_1's multi_logloss: 0.220121
Early stopping, best iteration is:
[131]	training's multi_logloss: 0.0330821	valid_1's multi_logloss: 0.215393
0 val f1 0.89348631493073
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.0554685	valid_1's multi_logloss: 0.225007
[200]	training's multi_logloss: 0.0112911	valid_1's multi_logloss: 0.222784
Early stopping, best iteration is:
[144]	training's multi_logloss: 0.0272102	valid_1's multi_logloss: 0.218445
1 val f1 0.8941891662225162
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.0565409	valid_1's multi_logloss: 0.240719
[200]	training's multi_logloss: 0.0115091	valid_1's multi_logloss: 0.231806
Early stopping, best iteration is:
[138]	training's multi_logloss: 0.0303208	valid

In [9]:
oof = np.argmax(oof, axis=1)
print('oof f1', metrics.f1_score(oof, y, average='macro'))

# 0.8866228588851547 - corr, v_count_0.5, v_count_tuo [0.7, 6.5, 20] , v_corr_tuo      
# 0.8874926343083475 - v_count_0.5, v_count_tuo [0, 2, 6, 20]
# 0.8884029972976388 - v_count_0.5, v_count_tuo [0, 2, 6, 20], median
# 0.8939494737221377 - v_count_0.5, v_count_tuo [2, 6, 20], median                   *
# 0.890917528266248  - v_count_0.5, v_count_tuo [2, 6], median

oof f1 0.8923409440176027


In [10]:
pred = np.argmax(pred, axis=1)
sub = test_label[['ship']]
sub['pred'] = pred

print(sub['pred'].value_counts(1))
sub['pred'] = sub['pred'].map(type_map_rev)
#sub.to_csv('result.csv', index=None, header=None)

0    0.629
1    0.242
2    0.129
Name: pred, dtype: float64


In [11]:
ret = []
for index, model in enumerate(models):
    df = pd.DataFrame()
    df['name'] = model.feature_name()
    df['score'] = model.feature_importance()
    df['fold'] = index
    ret.append(df)
    
df = pd.concat(ret)

In [12]:
df = df.groupby('name', as_index=False)['score'].mean()
df = df.sort_values(['score'], ascending=False)

In [13]:
df

,name,score
3,slope_2,615.0
43,y_max,561.0
38,x_median,500.6
44,y_max_x_min,490.2
39,x_min,468.4
36,x_max_y_min,416.6
5,v_cut_0.5_count_1.0,413.0
48,y_min,382.6
47,y_median,377.0
13,v_cut_0.5_count_4.0,373.2


In [14]:
sub.to_csv('C:/Users/f3107/Desktop/hy_data/result_0204.csv', index=None, header=None)